In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [75]:
#cargar archivo desde seaborn 
df=pd.read_csv('Mexico_Datos_Limpios_2.csv')
df.head(5)

,last_scraped,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_location,license,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,Dolar
0,27/12/2024,Dici,28/06/2010,"Mexico City, Mexico",0,0,0,f,t,t,...,4.80,4.80,4.90,4.90,0,1.0,0.0,0,0.00,$196.50
1,28/12/2024,Fernando,09/08/2010,"Mexico City, Mexico",within an hour,92%,92%,t,t,t,...,4.56,4.70,4.87,4.98,0,4.0,1.0,0,0.40,$874.64
2,28/12/2024,Nicholas,04/01/2011,"Mexico City, Mexico",within a few hours,100%,56%,t,t,t,...,4.82,4.76,4.94,4.98,0,2.0,0.0,0,0.31,$103.16
3,27/12/2024,Trisha,24/08/2010,"Mexico City, Mexico",within an hour,100%,94%,t,t,t,...,4.91,4.96,4.96,4.96,0,2.0,1.0,0,0.84,$60.06
4,28/12/2024,Fernando,31/05/2011,"Mexico City, Mexico",0,0,0,f,t,f,...,4.80,4.80,4.90,4.90,0,1.0,2.0,0,0.00,$0.00


In [77]:
#cargar archivo csv desde seaboarn
#df=sns.load_dataset(name='titanic')
#df.head(5)

In [78]:
# Relleno de valores nulos 

df = df.fillna(method="bfill")
df = df.fillna(method="ffill")

C:\Users\ghseb\AppData\Local\Temp\ipykernel_14652\2761572045.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="bfill")
C:\Users\ghseb\AppData\Local\Temp\ipykernel_14652\2761572045.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")


In [79]:
# Asegúrate de que tienes un DataFrame llamado 'df'
print(df.columns)

Index(['last_scraped', 'host_name', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
       'property_type', 'room_type', 'bathrooms_text', 'price',
       'has_availability', 'calendar_last_scraped', 'first_review',
       'last_review', 'instant_bookable', 'id', 'host_id',
       'host_listings_count', 'host_total_listings_count', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30',
       'availability_90', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_loca

In [80]:
#f1=df

In [81]:
# Imprimir las columnas que son de tipo numérico
numeric_columns = df.select_dtypes(include=['number']).columns
print(numeric_columns)

Index(['id', 'host_id', 'host_listings_count', 'host_total_listings_count',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30',
       'availability_90', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_location', 'license',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')


In [82]:
df['price'] = df['price'].astype(str).str.replace(r'[\$,]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)

In [83]:
print(df['price'].head())

0     4044.0
1    18000.0
2     2123.0
3     1236.0
4        0.0
Name: price, dtype: float64


In [84]:
#convertimos string a tipos numéricos
#df['host_is_superhost'] = df['host_is_superhost'].replace({'f': 0, 't': 1})

In [85]:
# Convertir variables categóricas a numéricas
#df['host_is_superhost'] = df['host_is_superhost'].map({'f': 0, 't': 1})
#df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'f': 0, 't': 1})
#df['host_identity_verified'] = df['host_identity_verified'].map({'f': 0, 't': 1})
#df['instant_bookable'] = df['instant_bookable'].map({'f': 0, 't': 1})

In [86]:
print(df['host_is_superhost'].head(10))

0    f
1    t
2    t
3    t
4    f
5    t
6    t
7    t
8    t
9    t
Name: host_is_superhost, dtype: object


In [87]:
# Declaramos las variables dependientes e independientes para la regresión logística
#Vars_Indep = df[['pclass', 'age', 'fare']]
#Var_Dep = df['alive']
Vars_Indep1= df[['price', 'accommodates']]
Var_Dep1= df['host_is_superhost']

# Caso 1
Vars_Indep2 = df[['price', 'accommodates']]
Var_Dep2 = df['host_is_superhost']

# Caso 2
Vars_Indep3 = df[['bedrooms', 'beds']]
Var_Dep3 = df['instant_bookable']

# Caso 3
Vars_Indep4 = df[['price', 'availability_90']]
Var_Dep4 = df['host_identity_verified']

# Caso 4
Vars_Indep5 = df[['minimum_nights', 'reviews_per_month']]
Var_Dep5 = df['host_is_superhost']

# Caso 5
Vars_Indep6 = df[['accommodates', 'bathrooms']]
Var_Dep6 = df['instant_bookable']

# Caso 6
Vars_Indep7 = df[['price', 'number_of_reviews']]
Var_Dep7 = df['host_identity_verified']

# Caso 7
Vars_Indep8 = df[['beds', 'availability_30']]
Var_Dep8 = df['host_is_superhost']

# Caso 8
Vars_Indep9 = df[['price', 'review_scores_rating']]
Var_Dep9 = df['instant_bookable']

# Caso 9
Vars_Indep10 = df[['reviews_per_month', 'bedrooms']]
Var_Dep10 = df['host_identity_verified']

In [88]:
# Redefinimos las variables
X = Vars_Indep1
y = Var_Dep1
x2 = Vars_Indep2
y2 = Var_Dep2           
x3 = Vars_Indep3
y3 = Var_Dep3
x4 = Vars_Indep4
y4 = Var_Dep4
x5 = Vars_Indep5
y5 = Var_Dep5
x6 = Vars_Indep6
y6 = Var_Dep6
x7 = Vars_Indep7
y7 = Var_Dep7
x8 = Vars_Indep8
y8 = Var_Dep8
x9 = Vars_Indep9
y9 = Var_Dep9
x10 = Vars_Indep10
y10 = Var_Dep10


In [89]:
# Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)



# Dividir cada conjunto de variables independientes y dependientes
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size=0.3, random_state=None)
X2_train, X2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.3, random_state=None)
X3_train, X3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.3, random_state=None)
X4_train, X4_test, y4_train, y4_test = train_test_split(x4, y4, test_size=0.3, random_state=None)
X5_train, X5_test, y5_train, y5_test = train_test_split(x5, y5, test_size=0.3, random_state=None)
X6_train, X6_test, y6_train, y6_test = train_test_split(x6, y6, test_size=0.3, random_state=None)
X7_train, X7_test, y7_train, y7_test = train_test_split(x7, y7, test_size=0.3, random_state=None)
X8_train, X8_test, y8_train, y8_test = train_test_split(x8, y8, test_size=0.3, random_state=None)
X9_train, X9_test, y9_train, y9_test = train_test_split(x9, y9, test_size=0.3, random_state=None)
X10_train, X10_test, y10_train, y10_test = train_test_split(x10, y10, test_size=0.3, random_state=None)

In [90]:
#se escalan todos los datos
escalar = StandardScaler()

In [91]:
# Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
#X_train = escalar.fit_transform(X_train)
#X_test = escalar.transform(X_test)

# Escalamos cada conjunto de datos (X_train y X_test) para las 10 variables
# --- Conjunto 1 ---
X1_train = escalar.fit_transform(X1_train)  
X1_test = escalar.transform(X1_test)       

# --- Conjunto 2 ---
X2_train = escalar.fit_transform(X2_train)
X2_test = escalar.transform(X2_test)

# --- Conjunto 3 ---
X3_train = escalar.fit_transform(X3_train)
X3_test = escalar.transform(X3_test)

# --- Conjunto 4 ---
X4_train = escalar.fit_transform(X4_train)
X4_test = escalar.transform(X4_test)

# --- Conjunto 5 ---
X5_train = escalar.fit_transform(X5_train)
X5_test = escalar.transform(X5_test)

# --- Conjunto 6 ---
X6_train = escalar.fit_transform(X6_train)
X6_test = escalar.transform(X6_test)

# --- Conjunto 7 ---
X7_train = escalar.fit_transform(X7_train)
X7_test = escalar.transform(X7_test)

# --- Conjunto 8 ---
X8_train = escalar.fit_transform(X8_train)
X8_test = escalar.transform(X8_test)

# --- Conjunto 9 ---
X9_train = escalar.fit_transform(X9_train)
X9_test = escalar.transform(X9_test)

# --- Conjunto 10 ---
X10_train = escalar.fit_transform(X10_train)
X10_test = escalar.transform(X10_test)

In [92]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()    

In [93]:
#entremanos el modelo   
#algoritmo.fit(X_train, y_train) 

# Entrenamos los 10 modelos uno por uno:

# Modelo 1
algoritmo.fit(X1_train, y1_train)

# Modelo 2
algoritmo.fit(X2_train, y2_train)

# Modelo 3
algoritmo.fit(X3_train, y3_train)

# Modelo 4
algoritmo.fit(X4_train, y4_train)

# Modelo 5
algoritmo.fit(X5_train, y5_train)

# Modelo 6
algoritmo.fit(X6_train, y6_train)

# Modelo 7
algoritmo.fit(X7_train, y7_train)

# Modelo 8
algoritmo.fit(X8_train, y8_train)

# Modelo 9
algoritmo.fit(X9_train, y9_train)

# Modelo 10
algoritmo.fit(X10_train, y10_train)

LogisticRegression()

In [94]:
print("Dimensiones del conjunto de entrenamiento:", X1_train.shape)
print("Dimensiones de X1_test:", X1_test.shape)
print("Dimensiones de X2_test:", X2_test.shape)
# Repite para los demás conjuntos de prueba

Dimensiones del conjunto de entrenamiento: (18396, 2)
Dimensiones de X1_test: (7885, 2)
Dimensiones de X2_test: (7885, 2)


In [95]:
# Selecciona las primeras 2 columnas de cada conjunto de prueba
#X1_test = X1_test[:, :2]
#X2_test = X2_test[:, :2]
#X3_test = X3_test[:, :2]
#X4_test = X4_test[:, :2]
#X5_test = X5_test[:, :2]
#X6_test = X6_test[:, :2]
#X7_test = X7_test[:, :2]
#X8_test = X8_test[:, :2]
#X9_test = X9_test[:, :2]
#X10_test = X10_test[:, :2]

In [96]:
#realizamos una predicción  
#y_pred = algoritmo.predict(X_test)
#y_pred


# Predicción Modelo 1
y1_pred = algoritmo.predict(X1_test)

# Predicción Modelo 2
y2_pred = algoritmo.predict(X2_test)

# Predicción Modelo 3
y3_pred = algoritmo.predict(X3_test)

# Predicción Modelo 4
y4_pred = algoritmo.predict(X4_test)

# Predicción Modelo 5
y5_pred = algoritmo.predict(X5_test)

# Predicción Modelo 6
y6_pred = algoritmo.predict(X6_test)

# Predicción Modelo 7
y7_pred = algoritmo.predict(X7_test)

# Predicción Modelo 8
y8_pred = algoritmo.predict(X8_test)

# Predicción Modelo 9
y9_pred = algoritmo.predict(X9_test)

# Predicción Modelo 10
y10_pred = algoritmo.predict(X10_test)

# Imprimir las predicciones de cada modelo
print("Predicciones Modelo 1:", y1_pred)    
print("Predicciones Modelo 2:", y2_pred)
print("Predicciones Modelo 3:", y3_pred)
print("Predicciones Modelo 4:", y4_pred)
print("Predicciones Modelo 5:", y5_pred)
print("Predicciones Modelo 6:", y6_pred)
print("Predicciones Modelo 7:", y7_pred)
print("Predicciones Modelo 8:", y8_pred)
print("Predicciones Modelo 9:", y9_pred)
print("Predicciones Modelo 10:", y10_pred)


Predicciones Modelo 1: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 2: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 3: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 4: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 5: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 6: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 7: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 8: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 9: ['t' 't' 't' ... 't' 't' 't']
Predicciones Modelo 10: ['t' 't' 't' ... 't' 't' 't']


In [97]:

# Convierte las etiquetas reales a números
#y1_test = [0 if label == 'f' else 1 for label in y1_test]
#y2_test = [0 if label == 'f' else 1 for label in y2_test]
#y3_test = [0 if label == 'f' else 1 for label in y3_test]
#y4_test = [0 if label == 'f' else 1 for label in y4_test]
##y5_test = [0 if label == 'f' else 1 for label in y5_test]
#y6_test = [0 if label == 'f' else 1 for label in y6_test]
#y7_test = [0 if label == 'f' else 1 for label in y7_test]
#y8_test = [0 if label == 'f' else 1 for label in y8_test]
#y9_test = [0 if label == 'f' else 1 for label in y9_test]
#y10_test = [0 if label == 'f' else 1 for label in y10_test]


In [98]:
print("Etiquetas reales (y1_test):", y1_test[:10])
print("Etiquetas predichas (y1_pred):", y1_pred[:10])

Etiquetas reales (y1_test): 15218    t
22688    t
10120    t
14490    t
22035    f
10996    f
22904    f
6085     f
20108    t
767      f
Name: host_is_superhost, dtype: object
Etiquetas predichas (y1_pred): ['t' 't' 't' 't' 't' 't' 't' 't' 't' 't']


In [99]:
#verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
#matriz = confusion_matrix(y_test, y_pred)
#print('Matriz de Confusión:')
#print(matriz)


#from sklearn.metrics import confusion_matrix

# Matriz de confusión para cada modelo:

# --- Modelo 1 ---
matriz1 = confusion_matrix(y1_test, y1_pred)
print('\nMatriz de Confusión Modelo 1:')
print(matriz1)

# --- Modelo 2 ---
matriz2 = confusion_matrix(y2_test, y2_pred)
print('\nMatriz de Confusión Modelo 2:')
print(matriz2)

# --- Modelo 3 ---
matriz3 = confusion_matrix(y3_test, y3_pred)
print('\nMatriz de Confusión Modelo 3:')
print(matriz3)

# --- Modelo 4 ---
matriz4 = confusion_matrix(y4_test, y4_pred)
print('\nMatriz de Confusión Modelo 4:')
print(matriz4)

# --- Modelo 5 ---
matriz5 = confusion_matrix(y5_test, y5_pred)
print('\nMatriz de Confusión Modelo 5:')
print(matriz5)

# --- Modelo 6 ---
matriz6 = confusion_matrix(y6_test, y6_pred)
print('\nMatriz de Confusión Modelo 6:')
print(matriz6)

# --- Modelo 7 ---
matriz7 = confusion_matrix(y7_test, y7_pred)
print('\nMatriz de Confusión Modelo 7:')
print(matriz7)

# --- Modelo 8 ---
matriz8 = confusion_matrix(y8_test, y8_pred)
print('\nMatriz de Confusión Modelo 8:')
print(matriz8)

# --- Modelo 9 ---
matriz9 = confusion_matrix(y9_test, y9_pred)
print('\nMatriz de Confusión Modelo 9:')
print(matriz9)

# --- Modelo 10 ---
matriz10 = confusion_matrix(y10_test, y10_pred)
print('\nMatriz de Confusión Modelo 10:')
print(matriz10)


Matriz de Confusión Modelo 1:
[[   0    0  398]
 [   0    0 4470]
 [   0    0 3017]]

Matriz de Confusión Modelo 2:
[[   0    0  424]
 [   0    0 4519]
 [   0    0 2942]]

Matriz de Confusión Modelo 3:
[[   0 4779]
 [   0 3106]]

Matriz de Confusión Modelo 4:
[[   0    0    2]
 [   0    0  378]
 [   0    0 7505]]

Matriz de Confusión Modelo 5:
[[   0    0  409]
 [   0    0 4435]
 [   0    0 3041]]

Matriz de Confusión Modelo 6:
[[   0 4748]
 [   0 3137]]

Matriz de Confusión Modelo 7:
[[   0    0    1]
 [   0    0  383]
 [   0    0 7501]]

Matriz de Confusión Modelo 8:
[[   0    0  433]
 [   0    0 4450]
 [   0    0 3002]]

Matriz de Confusión Modelo 9:
[[   0 4796]
 [   0 3089]]

Matriz de Confusión Modelo 10:
[[   0    0    1]
 [   0    0  379]
 [   0    0 7505]]


In [100]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Configura el modelo con balanceo de clases y ajuste de umbral
modelo = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)

# Lista de tus pares (X_train, X_test, y_train, y_test) para los 10 modelos
datos_modelos = [
    (X1_train, X1_test, y1_train, y1_test),
    (X2_train, X2_test, y2_train, y2_test),
    (X3_train, X3_test, y3_train, y3_test),
    (X4_train, X4_test, y4_train, y4_test),
    (X5_train, X5_test, y5_train, y5_test),
    (X6_train, X6_test, y6_train, y6_test),
    (X7_train, X7_test, y7_train, y7_test),
    (X8_train, X8_test, y8_train, y8_test),
    (X9_train, X9_test, y9_train, y9_test),
    (X10_train, X10_test, y10_train, y10_test)
]

# Corregir las matrices de confusión para cada modelo
for i, (X_train, X_test, y_train, y_test) in enumerate(datos_modelos, start=1):
    # Entrenar el modelo
    modelo.fit(X_train, y_train)
    
    # Predecir probabilidades y ajustar el umbral (ej. 0.3 para priorizar 'f')
    y_pred_prob = modelo.predict_proba(X_test)[:, 1]  # Probabilidad de 'f'
    y_pred_ajustado = ['f' if prob >= 0.3 else 't' for prob in y_pred_prob]  # Umbral ajustable
    
    # Matriz de confusión corregida
    matriz = confusion_matrix(y_test, y_pred_ajustado)
    
    print(f"\n--- Matriz de Confusión Modelo {i} (Umbral=0.3) ---")
    print(matriz)


--- Matriz de Confusión Modelo 1 (Umbral=0.3) ---
[[   0  288  110]
 [   0 3591  879]
 [   0 2139  878]]

--- Matriz de Confusión Modelo 2 (Umbral=0.3) ---
[[   0  305  119]
 [   0 3681  838]
 [   0 2118  824]]

--- Matriz de Confusión Modelo 3 (Umbral=0.3) ---
[[4779    0]
 [3106    0]]

--- Matriz de Confusión Modelo 4 (Umbral=0.3) ---
[[   0    0    2]
 [   0  282   96]
 [   0 3533 3972]]

--- Matriz de Confusión Modelo 5 (Umbral=0.3) ---
[[   0  219  190]
 [   0 3439  996]
 [   0 1564 1477]]

--- Matriz de Confusión Modelo 6 (Umbral=0.3) ---
[[4748    0]
 [3137    0]]

--- Matriz de Confusión Modelo 7 (Umbral=0.3) ---
[[   0    0    1]
 [   0  308   75]
 [   0 4944 2557]]

--- Matriz de Confusión Modelo 8 (Umbral=0.3) ---
[[   0  305  128]
 [   0 3538  912]
 [   0 2054  948]]

--- Matriz de Confusión Modelo 9 (Umbral=0.3) ---
[[4796    0]
 [3089    0]]

--- Matriz de Confusión Modelo 10 (Umbral=0.3) ---
[[   0    0    1]
 [   0  228  151]
 [   0 3359 4146]]


In [101]:
y1_test = np.where(y1_test == 'No', 'f', y1_test)

In [102]:
print("Clases en y1_test:", np.unique(y1_test))
print("Clases en y1_pred:", np.unique(y1_pred))

Clases en y1_test: ['0' 'f' 't']
Clases en y1_pred: ['t']


In [103]:
for i in range(1, 11):  # Del Modelo 1 al 10
    y_test = globals()[f'y{i}_test']
    y_pred = globals()[f'y{i}_pred']
    print(f"\nModelo {i} - Clases en y_test:", np.unique(y_test))
    print(f"Modelo {i} - Clases en y_pred:", np.unique(y_pred))


Modelo 1 - Clases en y_test: ['0' 'f' 't']
Modelo 1 - Clases en y_pred: ['t']

Modelo 2 - Clases en y_test: ['0' 'f' 't']
Modelo 2 - Clases en y_pred: ['t']

Modelo 3 - Clases en y_test: ['f' 't']
Modelo 3 - Clases en y_pred: ['t']

Modelo 4 - Clases en y_test: ['0' 'f' 't']
Modelo 4 - Clases en y_pred: ['t']

Modelo 5 - Clases en y_test: ['0' 'f' 't']
Modelo 5 - Clases en y_pred: ['t']

Modelo 6 - Clases en y_test: ['f' 't']
Modelo 6 - Clases en y_pred: ['t']

Modelo 7 - Clases en y_test: ['0' 'f' 't']
Modelo 7 - Clases en y_pred: ['t']

Modelo 8 - Clases en y_test: ['0' 'f' 't']
Modelo 8 - Clases en y_pred: ['t']

Modelo 9 - Clases en y_test: ['f' 't']
Modelo 9 - Clases en y_pred: ['t']

Modelo 10 - Clases en y_test: ['0' 'f' 't']
Modelo 10 - Clases en y_pred: ['t']


In [104]:
from sklearn.metrics import precision_score
import numpy as np

def precision_binaria(y_true, y_pred, pos_label='t'):
    """Calcula la precisión considerando solo 't' y 'f' (ignora otras clases)."""
    # Filtra solo muestras donde y_true es 't' o 'f'
    mask = np.isin(y_true, ['t', 'f'])
    y_true_bin = y_true[mask]
    y_pred_bin = y_pred[mask]
    
    # Calcula la precisión binaria
    return precision_score(y_true_bin, y_pred_bin, average='binary', pos_label=pos_label)

# --- Ejemplo de uso para todos los modelos ---
for i in range(1, 11):
    y_test = globals()[f'y{i}_test']
    y_pred = globals()[f'y{i}_pred']
    precision = precision_binaria(y_test, y_pred)
    print(f'Precisión Modelo {i} (binaria t vs f):', precision)

Precisión Modelo 1 (binaria t vs f): 0.4029651395752638
Precisión Modelo 2 (binaria t vs f): 0.394317115668141
Precisión Modelo 3 (binaria t vs f): 0.3939124920735574
Precisión Modelo 4 (binaria t vs f): 0.9520487124191298
Precisión Modelo 5 (binaria t vs f): 0.40676832530765117
Precisión Modelo 6 (binaria t vs f): 0.39784400760938493
Precisión Modelo 7 (binaria t vs f): 0.9514205986808727
Precisión Modelo 8 (binaria t vs f): 0.4028448738593666
Precisión Modelo 9 (binaria t vs f): 0.3917564996829423
Precisión Modelo 10 (binaria t vs f): 0.9519279553526129


In [105]:
from sklearn.metrics import precision_score

# Precisión para cada modelo (clasificación binaria con etiquetas 't' y 'f'):

# --- Modelo 1 ---
precision1 = precision_score(y1_test, y1_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 1:', precision1)

# --- Modelo 2 ---
precision2 = precision_score(y2_test, y2_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 2:', precision2)

# --- Modelo 3 ---
precision3 = precision_score(y3_test, y3_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 3:', precision3)

# --- Modelo 4 ---
precision4 = precision_score(y4_test, y4_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 4:', precision4)

# --- Modelo 5 ---
precision5 = precision_score(y5_test, y5_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 5:', precision5)

# --- Modelo 6 ---
precision6 = precision_score(y6_test, y6_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 6:', precision6)

# --- Modelo 7 ---
precision7 = precision_score(y7_test, y7_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 7:', precision7)

# --- Modelo 8 ---
precision8 = precision_score(y8_test, y8_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 8:', precision8)

# --- Modelo 9 ---
precision9 = precision_score(y9_test, y9_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 9:', precision9)

# --- Modelo 10 ---
precision10 = precision_score(y10_test, y10_pred, average='binary', pos_label='t')
print('\nPrecisión Modelo 10:', precision10)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
# Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

#exactitud = accuracy_score(y_test, y_pred)
#print('Exactitud del modelo:')
#print(exactitud)


#from sklearn.metrics import accuracy_score

# Calculamos la exactitud para cada modelo:

# --- Modelo 1 ---
exactitud1 = accuracy_score(y1_test, y1_pred)
print('\nExactitud Modelo 1:', exactitud1)

# --- Modelo 2 ---
exactitud2 = accuracy_score(y2_test, y2_pred)
print('\nExactitud Modelo 2:', exactitud2)

# --- Modelo 3 ---
exactitud3 = accuracy_score(y3_test, y3_pred)
print('\nExactitud Modelo 3:', exactitud3)

# --- Modelo 4 ---
exactitud4 = accuracy_score(y4_test, y4_pred)
print('\nExactitud Modelo 4:', exactitud4)

# --- Modelo 5 ---
exactitud5 = accuracy_score(y5_test, y5_pred)
print('\nExactitud Modelo 5:', exactitud5)

# --- Modelo 6 ---
exactitud6 = accuracy_score(y6_test, y6_pred)
print('\nExactitud Modelo 6:', exactitud6)

# --- Modelo 7 ---
exactitud7 = accuracy_score(y7_test, y7_pred)
print('\nExactitud Modelo 7:', exactitud7)

# --- Modelo 8 ---
exactitud8 = accuracy_score(y8_test, y8_pred)
print('\nExactitud Modelo 8:', exactitud8)

# --- Modelo 9 ---
exactitud9 = accuracy_score(y9_test, y9_pred)
print('\nExactitud Modelo 9:', exactitud9)

# --- Modelo 10 ---
exactitud10 = accuracy_score(y10_test, y10_pred)
print('\nExactitud Modelo 10:', exactitud10)


Exactitud Modelo 1: 0.3842739378566899

Exactitud Modelo 2: 0.37983512999365887

Exactitud Modelo 3: 0.3918833227647432

Exactitud Modelo 4: 0.9505389980976537

Exactitud Modelo 5: 0.3968294229549778

Exactitud Modelo 6: 0.38833227647431834

Exactitud Modelo 7: 0.9520608750792644

Exactitud Modelo 8: 0.37387444514901713

Exactitud Modelo 9: 0.39860494610019026

Exactitud Modelo 10: 0.952568167406468


In [ ]:
# Verificar la distribución de clases en y_test y y_pred
from collections import Counter

print("Distribución en y1_test:", Counter(y1_test))
print("Distribución en y1_pred:", Counter(y1_pred))

Distribución en y1_test: Counter({'f': 4855, 't': 3030})
Distribución en y1_pred: Counter({'t': 7885})


In [ ]:
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.metrics import recall_score

# Definir el modelo con balanceo de clases (para manejar desequilibrio)
modelo = LogisticRegression(class_weight='balanced', random_state=42)

# Entrenar el modelo con tus datos del Modelo 1
modelo.fit(X1_train, y1_train)

# Obtener probabilidades para la clase 'f' (asumiento que es la clase minoritaria)
y1_pred_prob = modelo.predict_proba(X1_test)[:, 1]  # Probabilidades de la clase 'f'

# Ajustar el umbral de decisión (empieza con 0.3 y ajusta según necesidad)
umbral = 0.3  # Puedes probar con 0.4, 0.2, etc.
y1_pred_ajustado = ['f' if prob >= umbral else 't' for prob in y1_pred_prob]

# Verificar distribución de predicciones
print("\nDistribución en y1_test (real):", Counter(y1_test))
print("Distribución en y1_pred_ajustado:", Counter(y1_pred_ajustado))

# Calcular métricas
sensibilidad1_ajustada = recall_score(y1_test, y1_pred_ajustado, pos_label='f')
print("\nSensibilidad ajustada (Recall para 'f'):", sensibilidad1_ajustada)


Distribución en y1_test (real): Counter({'f': 4855, 't': 3030})
Distribución en y1_pred_ajustado: Counter({'f': 6104, 't': 1781})

Sensibilidad ajustada (Recall para 'f'): 0.8055612770339856


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score

# Modelo con balanceo de clases
modelo = LogisticRegression(class_weight='balanced', random_state=42)
modelo.fit(X1_train, y1_train)

# Predicciones con umbral ajustado
y1_pred_prob = modelo.predict_proba(X1_test)[:, 1]  # Probabilidad de 'f'
umbral = 0.3  # Ajusta según necesidad
y1_pred_ajustado = ['f' if prob >= umbral else 't' for prob in y1_pred_prob]

# Sensibilidad corregida
sensibilidad = recall_score(y1_test, y1_pred_ajustado, pos_label='f')
print("Sensibilidad ajustada:", sensibilidad)

Sensibilidad ajustada: 0.8055612770339856


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from collections import Counter

# Listas de datos de entrenamiento y prueba para cada modelo
X_train_list = [X1_train, X2_train, X3_train, X4_train, X5_train, X6_train, X7_train, X8_train, X9_train, X10_train]
y_train_list = [y1_train, y2_train, y3_train, y4_train, y5_train, y6_train, y7_train, y8_train, y9_train, y10_train]
X_test_list = [X1_test, X2_test, X3_test, X4_test, X5_test, X6_test, X7_test, X8_test, X9_test, X10_test]
y_test_list = [y1_test, y2_test, y3_test, y4_test, y5_test, y6_test, y7_test, y8_test, y9_test, y10_test]

# Umbral ajustado
umbral = 0.3

# Iterar sobre los modelos
for i, (X_train, y_train, X_test, y_test) in enumerate(zip(X_train_list, y_train_list, X_test_list, y_test_list), start=1):
    # Crear y entrenar el modelo
    modelo = LogisticRegression(class_weight='balanced', random_state=42)
    modelo.fit(X_train, y_train)
    
    # Predicciones con umbral ajustado
    y_pred_prob = modelo.predict_proba(X_test)[:, 1]  # Probabilidad de 'f'
    y_pred_ajustado = ['f' if prob >= umbral else 't' for prob in y_pred_prob]
    
  
    
    # Calcular sensibilidad
    sensibilidad = recall_score(y_test, y_pred_ajustado, pos_label='f')
    print(f"Sensibilidad Modelo {i}:", sensibilidad)

Sensibilidad Modelo 1: 0.8055612770339856


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
#calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

#sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="no")
#print('Sensibilidad del modelo:')
#print(sensibilidad)


#from sklearn.metrics import recall_score

# Sensibilidad para cada modelo (versión para clasificación binaria):
# Asumiendo que 'no' es tu etiqueta positiva para sensibilidad

# --- Modelo 1 ---
sensibilidad1 = recall_score(y1_test, y1_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 1 (Recall para clase "f"):', sensibilidad1)

# --- Modelo 2 ---
sensibilidad2 = recall_score(y2_test, y2_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 2 (Recall para clase "f"):', sensibilidad2)

# --- Modelo 3 ---
sensibilidad3 = recall_score(y3_test, y3_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 3 (Recall para clase "f"):', sensibilidad3)

# --- Modelo 4 ---
sensibilidad4 = recall_score(y4_test, y4_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 4 (Recall para clase "f"):', sensibilidad4)

# --- Modelo 5 ---
sensibilidad5 = recall_score(y5_test, y5_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 5 (Recall para clase "f"):', sensibilidad5)

# --- Modelo 6 ---
sensibilidad6 = recall_score(y6_test, y6_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 6 (Recall para clase "f"):', sensibilidad6)

# --- Modelo 7 ---
sensibilidad7 = recall_score(y7_test, y7_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 7 (Recall para clase "f"):', sensibilidad7)

# --- Modelo 8 ---
sensibilidad8 = recall_score(y8_test, y8_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 8 (Recall para clase "f"):', sensibilidad8)

# --- Modelo 9 ---
sensibilidad9 = recall_score(y9_test, y9_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 9 (Recall para clase "f"):', sensibilidad9)

# --- Modelo 10 ---
sensibilidad10 = recall_score(y10_test, y10_pred, average='binary', pos_label='f')
print('\nSensibilidad Modelo 10 (Recall para clase "f"):', sensibilidad10)


Sensibilidad Modelo 1 (Recall para clase "f"): 0.0

Sensibilidad Modelo 2 (Recall para clase "f"): 0.0

Sensibilidad Modelo 3 (Recall para clase "f"): 0.0

Sensibilidad Modelo 4 (Recall para clase "f"): 0.0

Sensibilidad Modelo 5 (Recall para clase "f"): 0.0

Sensibilidad Modelo 6 (Recall para clase "f"): 0.0

Sensibilidad Modelo 7 (Recall para clase "f"): 0.0

Sensibilidad Modelo 8 (Recall para clase "f"): 0.0

Sensibilidad Modelo 9 (Recall para clase "f"): 0.0

Sensibilidad Modelo 10 (Recall para clase "f"): 0.0


**reemplazar los valores**

In [ ]:
#verificar los valores sin repetir
#unico=np.unique(df['class'])
#unico

# Verificar los valores únicos en las variables dependientes definidas
variables_dependientes = [Var_Dep1, Var_Dep2, Var_Dep3, Var_Dep4, Var_Dep5, Var_Dep6, Var_Dep7, Var_Dep8, Var_Dep9, Var_Dep10]

for i, var_dep in enumerate(variables_dependientes, start=1):
    unico = np.unique(var_dep)
    print(f"Valores únicos en Var_Dep{i}: {unico}")

Valores únicos en Var_Dep1: ['f' 't']
Valores únicos en Var_Dep2: ['f' 't']
Valores únicos en Var_Dep3: ['f' 't']
Valores únicos en Var_Dep4: ['f' 't']
Valores únicos en Var_Dep5: ['f' 't']
Valores únicos en Var_Dep6: ['f' 't']
Valores únicos en Var_Dep7: ['f' 't']
Valores únicos en Var_Dep8: ['f' 't']
Valores únicos en Var_Dep9: ['f' 't']
Valores únicos en Var_Dep10: ['f' 't']
